In [1]:
2+2

4

In [239]:
selector = 'listings-wrapper xs-flex xs-flex-wrap xs-grid sm-grid-cols-2 lg-grid-cols-3 xl-grid-cols-4 xs-gap-2 sm-gap-3 xs-mx2- sm-mx0'

print(f'Copy html element with the selector,')
print(f'"{selector}"')

Copy html element with the selector,
"listings-wrapper xs-flex xs-flex-wrap xs-grid sm-grid-cols-2 lg-grid-cols-3 xl-grid-cols-4 xs-gap-2 sm-gap-3 xs-mx2- sm-mx0"


In [265]:
#final main df
final_cols = ['app_num','street_num','street', 'city', 'prov', 
              'district', 'latitude', 'longitude', 'sold_date', 
              'dom', 'sold_price', 'listing_price', 'beds','baths', 
              'size', 'mls', 'brokerage', 'listing_date', 'full_address']

main_df_path = 'main_data/main_results.csv'

empty_main_df = pd.DataFrame(columns = final_cols)
empty_main_df.to_csv(main_df_path,index=False)



columns = ['unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']

# duplicated df
main_df_path = 'review_data/duplicated_df.csv'

empty_main_df = pd.DataFrame(columns = columns)
empty_main_df.to_csv(main_df_path,index=False)

# temp data
main_df_path = 'temp/0_temp_prop_data.csv'

empty_main_df = pd.DataFrame(columns = columns)
empty_main_df.to_csv(main_df_path,index=False)

geo_columns = ['row_index', 'unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']

# check coordintes df
main_df_path = 'review_data/check_coordinates.csv'

empty_main_df = pd.DataFrame(columns = geo_columns)
empty_main_df.to_csv(main_df_path,index=False)

In [266]:
import os
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim


class extract_properties:
    
    def __init__(self):
        pass
        
    columns = ['unit_num', 'street', 'city', 'prov', 
               'district', 'latitude', 'longitude', 
               'sold_date', 'dom', 'sold_price',
               'listing_price', 'beds', 'baths', 
               'size', 'mls', 'brokerage']

        
    def to_dataframe(self, zolo_text, columns = columns):
        zolo_prop = zolo_text.split("data-propertyid")
        
        prop_data = pd.DataFrame(columns=columns)
        
        for prop in zolo_prop:
            if "MLS" in prop:
                row = {}
                details = prop.split("&")

                unit_num = details[13].split(';')[1]

                street = details[15].split(';')[1]

                city = details[19].split(';')[1]

                prov = details[23].split(';')[1]

                district = details[35].split(';')[1]

                #latitude
                latitude = details[42].split('=')[1].split()[0][1:-1]


                #longitude
                longitude = details[44].split('=')[1].split()[0][1:-1]

                # sold date
                sold_date = details[51].split('•')[1]

                # days on market
                dom = details[55].split('"p1">')[1]

                # sold price
                sold_price = details[63].split('$')[1]
                
                # listing price
                listing_price = details[67].split('$')[1]

                # num of beds
                beds = details[75].split(';')[1]

                # num of baths
                baths = details[79].split(';')[1]

                # size info may not be available, subsequent detail index will change
                # check if size detail exists in listing, 600-699 sqft or 1500-2000 sqft
                if ('99 sqft' in prop) or ('00 sqft' in prop):
                    size = details[83].split(';')[1]
                    mls = details[91].split(';')[1]
                    brokerage = details[99].split(';')[1]

                else:
                    size = np.NaN
                    mls = details[87].split(';')[1]
                    brokerage = details[95].split(';')[1]


                row = [unit_num, street, city, prov, district, latitude, longitude, 
                       sold_date, dom, sold_price, listing_price, beds, baths, size, mls, brokerage]

                row = pd.DataFrame([row], columns = columns)

                prop_data = pd.concat([prop_data, row])
                
        
        # remove all unwanted spaces in each columns
        def strip_string(value):
            # Check if the value is NaN (None) and return it as is
            if pd.isna(value):
                return value
            # Otherwise, apply the strip() method to remove leading and trailing whitespaces
            return value.strip()
        
        prop_data = prop_data.applymap(strip_string)   

        property_df = prop_data.reset_index(drop=True)
        
        print("Successfully converted to a dataframe")
        
        return property_df
    
    def clean_beds(self, property_df):
        # remove the bd suffixes
        property_df['beds'] = property_df['beds'].apply(lambda x: x.split()[0])
        print("Successfully extracted beds")
       
    
    def clean_baths(self, property_df):
        # remove the ba suffixes
        property_df['baths'] = property_df['baths'].apply(lambda x: x.split()[0])
        print("Successfully extracted baths")
        
    def clean_dom(self, property_df):
        
        empty_rows = None
        
        if '' in property_df['dom'].unique():
            empty_rows_index = property_df[property_df['dom']==''].index
            print(f'Check rows on website with empty DOMs is in fact 0: {empty_rows_index}')
            
            # replace empty rows with empty strings
            raw_df['dom'] = raw_df['dom'].replace('', '0 days on market')

        
        try:
            property_df['dom'] = property_df['dom'].apply(lambda x: x.split()[0])
            property_df['dom'] = pd.to_numeric(property_df['dom'])
            print("Successfully extracted DOMs with no concerns")
        
        except:
            print('DOMs has an unexpected value')
            
                
    def clean_sold_date(self, property_df):
        def get_sold_date(date, currentYear=2023):
            months_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 
                           'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
            month = date.split()[0]
            month_num = months_list.index(month)

            date_num = date.split()[1].split('th')[0]
            return pd.to_datetime(f'{currentYear}-{month_num+1}-{date_num}')

        property_df['sold_date'] = property_df['sold_date'].apply(lambda x: get_sold_date(x))
        
        print("Successfully extracted sold date")

    def clean_prices(self, property_df):
        
        """
        Remvoe the comma and change price column datatypes to numeric
        """
        def remove_comma(x):
            """
            Remove the comma in he prices
            Input: '625,000'
            Ouput: '625000'
            """
            number = ''
            for i in x.split(','):
                number += i
            return number
        
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(remove_comma)
            
        property_df[['sold_price','listing_price']] = property_df[['sold_price','listing_price']].applymap(pd.to_numeric)
        
        print("Successfully converted price data from string to numeric")
            
    ## make function, add_price_Difference
    def add_listing_date(self, property_df):
        # get listing date
        listing_date = property_df['sold_date'] - pd.to_timedelta(property_df['dom'],
                                                                  unit='days')
        # add to main dataset
        property_df['listing_date'] = listing_date
        print("Successfully calculated listing date")
        
    def estimate_size(self, property_df):
        # clean unit sizes
        size_range = property_df['size'].apply( lambda x: x.split()[0] if type(x) == str else x)

        # get average size of range
        size = size_range.apply(lambda x: ( int(x.split('-')[0]) + int(x.split('-')[1]) ) // 2 if type(x) == str else x )

        # round off size to end in 00.0, ie. 500
        size = size.apply(lambda x: x + 1 if x%2 == 1 else x)
        size = size.astype('Int64')

        property_df['size'] = size
        print("Successfully calculated size")
        
    def add_street_num(self, property_df):
        # get the street number
        property_df['street_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[-1])
        print("Successfully extracted street number")

    def add_app_num(self, property_df):
        # if has no apartment number give the appartment number -999
        property_df['app_num'] = property_df['unit_num'].apply(lambda x: x.split('-')[0] if len(x.split('-'))>1 else -999)
        print("Successfully extracted appartment number")
        
    def add_full_address(self, property_df):
        # get lat and long coordinates from full address
        property_df['full_address'] = property_df[['street_num','street','city','prov']].apply(lambda row: ' '.join(row), axis=1)
        print("Successfully extracted full address")
        
    global temp_data_path
    temp_data_path = 'temp/0_temp_prop_data.csv'
    
    def temp_data(self, property_df):
        """
        Store the property data in a temporary folder
        
        Useful for addressing wrong coordinates and duplicates
        """
        # save current df to temp
        property_df.to_csv(temp_data_path, index=False)
        
        
        
    def get_coordinates(self, address):
            """
            Helper function for function check_geo_coordinates to get latitude and longitude coordinates 
            from the library geopy.
            Input the address and return the latitude and longitude.
            """
            # Initialize a geocoder with the Nominatim service
            geolocator = Nominatim(user_agent="forward_geocoder")

            # Use the geocoder to get the location details (longitude and latitude)
            location = geolocator.geocode(address)

            if location:
                latitude = location.latitude
                longitude = location.longitude
                return latitude, longitude
            
        
    def check_geo_coordinates(self, property_df):
        """
        This function compares the zolo coordinates and geopy coorindates.
        If the sets of coordinates differ by 0.1 degree in either direction, 
            then return those rows for further review.
            
        0.001 degree is approximately 0.111km = 111 meters
        """
        
        # call on another class function to get coorindates
        geo_coords = pd.DataFrame(tuple(property_df['full_address'].apply(self.get_coordinates )), columns=['lat','long'])
            
        # update latitude and longitude data type
        property_df[['latitude','longitude']] = property_df[['latitude','longitude']].apply(pd.to_numeric)

        # compare the coordinates
        coords_df =  pd.DataFrame(columns =['lat_diff', 'long_diff'])

        coords_df['lat_diff'] = np.abs(property_df['latitude'] - geo_coords['lat'])
        coords_df['long_diff'] = np.abs(property_df['longitude'] - geo_coords['long'])

        # 0.01 diff in lat is 1.11km 
        rows_to_check = coords_df[(coords_df['lat_diff'] > 0.01) | (coords_df['long_diff'] > 0.01)].index
        
        # df of rows to check
        check_geo_df = property_df.iloc[rows_to_check]
        print(check_geo_df.iloc[0])
        
        check_geo_df = check_geo_df.reset_index()
        print(check_geo_df.iloc[0])

        check_geo_df = check_geo_df.rename(columns={'index':'row_index'})
        
        
        # master df of geo issues
        check_geo_df_path = 'review_data/check_coordinates.csv'
        main_check_geo_df = pd.read_csv(check_geo_df_path)
        
        main_check_geo_df = self.concat_df(main_check_geo_df, check_geo_df)
        
        main_check_geo_df.to_csv(check_geo_df_path, index=False)
        
        print("Completed adding rows that needs to be checked for coordinate issues")
        
        
    # final columns
    final_cols = ['app_num','street_num','street', 'city', 'prov', 
              'district', 'latitude', 'longitude', 'sold_date', 
              'dom', 'sold_price', 'listing_price', 'beds','baths', 
              'size', 'mls', 'brokerage', 'listing_date', 'full_address']
    
    main_df_path = 'main_data/main_results.csv'
      
    global main_df
    main_df = pd.read_csv(main_df_path)
    

    def important_cols(self, df):
        """
        df is the processed data and we want to reduce the columns to only the cleaned columns
        """
        return df[final_cols]
         
    def check_duplicates(self, new_df):
        """
        Input:
            main_df: master df
            new_df: newly created df
            
        Checks the full address and MLS number for duplicates.
        If there are duplicates, store them if a different database for manual review.
        """
        
        temp_df = pd.read_csv(temp_data_path)
        
        temp_new_df = self.concat_df(temp_df, new_df)
        # get rows that are duplicates
        duplicated_df = temp_new_df[temp_new_df.duplicated()]
        
        print(f"Duplicate df shape is {duplicated_df.shape}")
        
        dup_df_path = 'review_data/duplicated_df.csv'
            
        dup_df = pd.read_csv(dup_df_path)
        
        # add new rows to main dup df
        dup_df = self.concat_df(dup_df, duplicated_df)
        
        # save 
        dup_df.to_csv(dup_df_path)
        
        print("Completed duplicate check")
            
    
    def concat_df(self, main_df,new_df):
        """
        Input: main dataframe, newly processed dataframe
        Returns: combined dataframe for a new main df
        """
        
        return pd.concat([main_df,new_df])
    
    def temp_to_main(self):
        temp_data_path = 'temp/temp_prop_data.csv'
        
        temp_df = pd.read_csv("temp_data_path")
        
        temp_df = self.important_cols(temp_df)
        new_df = self.important_cols(new_df)
            
        concat_df(main_df, new_df).to_csv(main_df_path, index=False)
        
        print("Added new df to main")
        
    def clear_rows(self,df):
        """
        Clear rows but keep the same column structure.
        
        Used to clear
            1. review geo coords table
            2. review duplicates table
            3. occasionally temp table
        """
        
        return df[0:0]

## Start workflow

In [267]:
html_file = 'raw_data/zolo.html'

with open(html_file, "r", encoding='utf-8') as f:
    text= f.read()

In [268]:
apple = extract_properties()
raw_df = apple.to_dataframe(text)
apple.clean_beds(raw_df)
apple.clean_baths(raw_df)
apple.clean_dom(raw_df)
apple.clean_sold_date(raw_df)
apple.clean_prices(raw_df)
apple.add_listing_date(raw_df)
apple.estimate_size(raw_df)
apple.add_street_num(raw_df)
apple.add_app_num(raw_df)
apple.add_full_address(raw_df)
apple.temp_data(raw_df)

# may require manual review because the street name is 
    # 1. does not match with an address in the geopy.geocoders database
    # 2. Wrong
        # 2.1 spelling error
        # 2.2 missing information
apple.check_geo_coordinates(raw_df)
apple.check_duplicates(raw_df)

Successfully converted to a dataframe
Successfully extracted beds
Successfully extracted baths
Successfully extracted DOMs with no concerns
Successfully extracted sold date
Successfully converted price data from string to numeric
Successfully calculated listing date
Successfully calculated size
Successfully extracted street number
Successfully extracted appartment number
Successfully extracted full address
unit_num                                                    57-847
street                                             Sheppard Avenue
city                                                       Toronto
prov                                                            ON
district                                              Clanton Park
latitude                                                       0.0
longitude                                                      0.0
sold_date                                      2023-07-28 00:00:00
dom                                                   

In [11]:
# correct the coordinates with the manually reviewed address
# addreseses below are only for zolo.html file

if html_file == 'raw_data/zolo.html':
    correct_coords_1 = apple.get_coordinates('847 Sheppard Avenue W Toronto ON')
    correct_coords_2 = apple.get_coordinates('66 Forest Manor Road Toronto ON')

    raw_df.loc[1, ['latitude','longitude']]=correct_coords_1
    raw_df.loc[1, ['latitude','longitude']]=correct_coords_2
    raw_df

In [260]:
raw_df['size'].isna().sum()

4

In [254]:
orange = extract_properties()
temp_df = pd.read_csv("temp/temp_prop_data.csv")
